In [63]:
import os
import xml.etree.ElementTree as ET

All tags:
{'link', 'plate_number_image_url', 'plate_id', 'plate_title', 'tags', 'plate_region', 'fon_id', 'fon_title', 'model', 'photo_url', 'plate_number', 'country', 'model2', 'car'}

In [60]:
necessary_tags = ("car",
                  "model",
                  "model2"  # Model generation
                  )

In [106]:
def get_gen_folder_name(generation: str) -> str:
    """
    Generation name looks like a mess. This func will extract only numeric info
    :param generation:
    :return: number or string like "2nd"
    """

    try:
        int(generation)
        return generation
    except ValueError:
        gen = generation.split()[0]
        if gen[-1] == ',':
            gen = gen[:-1]
        return gen

In [107]:
def remove_slash(model:str) -> str:
    """
    We don't need lots of nested folders!
    """
    return model.replace("/", "_&_")

In [108]:

def parse_xml(file_path):
    PATH = "папка/"
    with open(file_path, 'r', encoding='utf-8') as file:
        tree = ET.parse(file)
    root = tree.getroot()

    # Access elements and attributes in the XML file
    for n, child in enumerate(root[:100]):


        tags = []
        for subchild in child:
            # if subchild.tag == "car":
            #     if subchild.text != "UAZ":
            #         break
            # else:
            #     if subchild.tag != "country":
            #         print(subchild.tag, subchild.text)
            # continue




            if subchild.tag in necessary_tags:
                # Our necessary_tags is ordered sequence, so we don't worry about values matching
                if subchild.tag == "model2":
                    text = get_gen_folder_name(subchild.text)
                    # print(text)
                # elif subchild.tag == "model2":
                #     text = subchild.text
                #     if type(text) != str:
                #         text = '_'.join(text)
                else:
                    text = subchild.text
                text = remove_slash(text)
                tags.append(text)

        os.makedirs(PATH + '/'.join(tags), exist_ok=True)

            # if subchild.tag == 'photo_url':
            #     print(subchild.text)


In [109]:

# Example usage
file_path = '../50k/exp_am_1line.xml'
parse_xml(file_path)